# Aula 14 - Cálculo de PCM usando PySCF

Autor: [Prof. Elvis do A. Soares](https://github.com/elvissoares) 

Contato: [elvis@peq.coppe.ufrj.br](mailto:elvis@peq.coppe.ufrj.br) - [Programa de Engenharia Química, PEQ/COPPE, UFRJ, Brasil](https://www.peq.coppe.ufrj.br/)

---

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from pyscf import gto, dft
from pyscf.solvent import pcm
from pyscf.geomopt import geometric_solver
from pyscf.tools import cubegen # para exportar aquivos cube

Baseado em https://github.com/pyscf/pyscf/blob/master/examples/solvent/05-pcm.py

# Glicina e tautômeros

## Otimizando a geometria no vácuo

Definindo a geometria inicial com conjunto de base

In [ ]:
aminoacid = gto.M(atom='''
        N    0.77595  -0.38543   0.40918
        C   -1.47791   0.45532  -0.23939
        C   -0.64608  -0.28047   0.79178
        O   -2.61517   0.14434  -0.52409
        O   -0.85915   1.46529  -0.79124
        H    0.04515   1.46444  -0.42620
        H   -1.07166  -1.28155   0.90583
        H   -0.73742   0.26337   1.73609
        H    0.85780  -0.88722  -0.47349
        H    1.28537  -0.92052   1.10915''',
              basis='6-31+g*', verbose=1)

zwitterion = gto.M(atom='''
        N   0.80069   0.95989  -0.01502
        C   2.28229   1.04570  -0.01205
        C   2.75010   1.30567  -1.41047
        O   1.89488   1.39987  -2.26935
        O   3.94347   1.40102  -1.57562
        H   0.41459   0.78110   0.92045
        H   0.53515   0.21886  -0.68231
        H   0.44056   1.84213  -0.41111
        H   2.75669   0.11558   0.31025
        H   2.65463   1.86920   0.60213''',
              basis='6-31+g*', verbose=1)

Definindo qual molécula iremos usar

In [ ]:
mol = zwitterion
mol_string = 'glycine_zwitterion'  # 'glycine_aminoacid' ou 'glycine_zwitterion'

Definindo qual o funcional de DFT iremos usar

In [ ]:
mf = dft.RKS(mol,xc='b3lyp')
new_mol = geometric_solver.optimize(mf)

Revelando as novas coordenadas

In [ ]:
new_mol.atom_coords(unit='Angstrom')

Colocando as novas coordenadas na molécula geometria original

In [ ]:
mol.set_geom_(new_mol.atom_coords(unit='Angstrom'))

Fazendo cálculo de densidade eletrônica e energia

In [ ]:
mf.kernel()

e0 = mf.e_tot

Calculando a densidade eletrônica

In [ ]:
dm = mf.make_rdm1() # densidade eletrônica na presença do solvente

Exportando a densidade eletrônica

In [ ]:
cubegen.density(mol, f'rho_{mol_string}.cube', dm)

Exportando o potencial eletrostático 

In [ ]:
cubegen.mep(mol, f'esp_{mol_string}.cube', dm)

## Otimizando a geometria usando modelo PCM para solvente

Definindo solvente e modelo PCM

In [ ]:
cm = pcm.PCM(mol)
cm.eps = 78.3553  # constante dielétrica da água
cm.method = 'SS(V)PE' # or IEF-PCM, C-PCM, SS(V)PE, COSMO, see https://manual.q-chem.com/5.4/topic_pcm-em.html
cm.lebedev_order = 29 # lebedev grids on the cavity surface, lebedev_order=29  <--> # of grids = 302

Otimizando a geometria agora na presença do solvente

In [ ]:
mf_sol = dft.RKS(mol, xc='b3lyp').PCM(cm)
mol_solvent = geometric_solver.optimize(mf_sol)

Novas coordenadas após otimização 

In [ ]:
mol_solvent.atom_coords(unit='Angstrom')

Fazendo cálculo de densidade eletrônica e energia

In [ ]:
mf_sol.kernel()

e_sol = mf_sol.e_tot

Cálculando a matriz densidade

In [ ]:
dm = mf_sol.make_rdm1() # densidade eletrônica na presença do solvente

Exportando a densidade eletrônica em solvente

In [ ]:
cubegen.density(mol_solvent, f'rho_{mol_string}_inwater.cube', dm)

Exportando o potencial eletrostático em solvente

In [ ]:
cubegen.mep(mol_solvent, f'esp_{mol_string}_inwater.cube', dm)

Cálculo da contribuição eletrostática da energia de solvatação

In [ ]:
Gelst = (e_sol-e0) * 627.5095  # convert from Hartree to kcal/mol
print(f'Contribuição eletrostática da energia de solvatação: {Gelst:.2f} kcal/mol')